# Обучаем модели для приложения

In [27]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import numpy as np

In [28]:
df_salary_ML = pd.read_csv('C:\\Users\\velvi\\Marta\\ITMO\\projects\\graduate work\\data\\df_vac_not_skills_7626.csv', sep=';')

In [29]:
df_salary_ML.head(1)

,website,city,subjects_RF,city_latitude,city_longitude,profession_category,specialization,job_title,description,work_schedule,experience,education,salary_from,salary_up,company,date_of_publication,link
0,superjob,Химки,Московская область,55.889361,37.444853,Разработчики,"Программист, разработчик",Программист-разработчик 1С,"<p><b>Обязанности:</b></p><ul><li>Внедрение, а...",полный рабочий день,от 1 года,не имеет значения,96257,102999,Дом Моды HENDERSON,2025-05-05,https://www.superjob.ru/vacancy/50134568


In [30]:
# Обучение модели ML, для заполнения колонки "salary_from"
df = df_salary_ML.copy()

# Обрезаем выбросы
q_low = df['salary_from'].quantile(0.01)
q_high = df['salary_from'].quantile(0.99)
df = df[(df['salary_from'] >= q_low) & (df['salary_from'] <= q_high)]
# q_high = 300000
# df = df[df['salary_from'] <= q_high]

# Признаки
cat_features_names = ['city', 'profession_category', 'specialization',
                      'experience', 'work_schedule', 'education']
num_features_names = ['salary_up']

# Формируем X и y
X = df[num_features_names + cat_features_names]
y = df['salary_from']

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучающая и тестовая выборки
train_pool = Pool(X_train, y_train, cat_features=cat_features_names)
test_pool = Pool(X_test, y_test, cat_features=cat_features_names)

# Модель
model_ML_salary = CatBoostRegressor(loss_function='RMSE', verbose=0)

# Подбор гиперпараметров
grid = {
    'depth': [3, 6, 8],
    'iterations': [500, 700, 1600],
    'learning_rate': [0.05, 0.1],
}

grid_search_result = model_ML_salary.grid_search(grid, X=train_pool, cv=3, plot=False)

# Предсказания
y_pred_train = model_ML_salary.predict(X_train)
y_pred_test = model_ML_salary.predict(X_test)

# Метрики
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f'R² на обучающей выборке: {r2_train:.4f}')
print(f'R² на тестовой выборке: {r2_test:.4f}')
print(f'RMSE на обучающей выборке: {rmse_train:.2f}')
print(f'RMSE на тестовой выборке: {rmse_test:.2f}')

# Итоговая метрика
y_pred = model_ML_salary.predict(X_test)
rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)))

# Лучшие параметры
best_params = grid_search_result['params']
print("Лучшие параметры:", best_params)


bestTest = 18059.83133
bestIteration = 440

0:	loss: 18059.8313252	best: 18059.8313252 (0)	total: 13.9s	remaining: 3m 56s

bestTest = 17895.33452
bestIteration = 498

1:	loss: 17895.3345161	best: 17895.3345161 (1)	total: 26.2s	remaining: 3m 29s

bestTest = 17991.06298
bestIteration = 682

2:	loss: 17991.0629818	best: 17895.3345161 (1)	total: 42.2s	remaining: 3m 30s

bestTest = 17682.6573
bestIteration = 661

3:	loss: 17682.6572975	best: 17682.6572975 (3)	total: 59.2s	remaining: 3m 27s

bestTest = 17790.338
bestIteration = 1444

4:	loss: 17790.3380023	best: 17682.6572975 (3)	total: 1m 40s	remaining: 4m 20s

bestTest = 17588.19875
bestIteration = 843

5:	loss: 17588.1987494	best: 17588.1987494 (5)	total: 2m 20s	remaining: 4m 41s

bestTest = 17599.89947
bestIteration = 499

6:	loss: 17599.8994746	best: 17588.1987494 (5)	total: 2m 46s	remaining: 4m 20s

bestTest = 17496.80752
bestIteration = 499

7:	loss: 17496.8075161	best: 17496.8075161 (7)	total: 3m 11s	remaining: 3m 59s

bestTest = 17

In [ ]:
# df[['salary_from', 'salary_up']].corr()


In [ ]:
# df[df['experience'] == 'без опыта']['salary_from'].describe()


In [ ]:
# top_20 = (
#     df[df['experience'] == 'без опыта']
#     .sort_values('salary_from', ascending=False)
#     .head(20)
#     [['job_title', 'salary_from', 'company', 'city', 'link']]
# )
# top_20

In [ ]:
# df[df['experience'] == 'без опыта'].describe()

In [12]:
df

,website,city,subjects_RF,city_latitude,city_longitude,profession_category,specialization,job_title,description,work_schedule,experience,education,salary_from,salary_up,company,date_of_publication,link
0,superjob,Химки,Московская область,55.889361,37.444853,Разработчики,"Программист, разработчик",Программист-разработчик 1С,"<p><b>Обязанности:</b></p><ul><li>Внедрение, а...",полный рабочий день,от 1 года,не имеет значения,96257,102999,Дом Моды HENDERSON,2025-05-05,https://www.superjob.ru/vacancy/50134568
1,superjob,Москва,Москва,55.755864,37.617698,Разработчики,"Программист, разработчик",Руководитель группы разработчиков 1С,<p><b>Обязанности:</b></p><ul><li>Управление к...,полный рабочий день,от 1 года,высшее,300000,380000,"ГКУ МО ""Централизованная бухгалтерия Московско...",2025-05-05,https://www.superjob.ru/vacancy/50362238
2,superjob,Сергиев Посад,Московская область,56.315311,38.135972,Разработчики,"Программист, разработчик",Инженер-технолог-программист (металлообработка...,<b>Обязанности:</b><p>- Технологическая подгот...,сменный график работы,от 3 лет,высшее,120000,120000,МЕТТОЙЛ,2025-05-05,https://www.superjob.ru/vacancy/30945389
3,superjob,Москва,Москва,55.755864,37.617698,Разработчики,"Программист, разработчик","Разработчик автохимии, лакокрасочных материалов",<p><b>Обязанности:</b><br />Требуется разработ...,неполный рабочий день,от 6 лет,высшее,70000,90000,ГК ПРОМТЕХ,2025-05-05,https://www.superjob.ru/vacancy/27748567
4,superjob,Хабаровск,Хабаровск,48.480229,135.071917,Разработчики,"Программист, разработчик",Ведущий программист 1С (ХА),<b>Обязанности:</b><ul><li>Администрирование\n...,полный рабочий день,от 1 года,не имеет значения,103180,103180,Хабаровский аэропорт,2025-05-04,https://www.superjob.ru/vacancy/41502773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7621,hh.ru,Москва,Москва,55.755864,37.617698,Прочие,Технический писатель,Технический писатель (Тераплан),<p><strong>Тераплан</strong> — это платформа д...,удаленная работа,от 1 года,не имеет значения,83940,110512,YADRO,2025-04-05,https://hh.ru/vacancy/119190416
7622,hh.ru,Краснодар,Краснодар,45.035470,38.975313,Прочие,Технический писатель,Технический писатель (1С),<p><strong>Обязанности:</strong></p> <ul> <li>...,удаленная работа,от 1 года,не имеет значения,79799,108577,ИК СИБИНТЕК,2025-04-14,https://hh.ru/vacancy/113041936
7623,hh.ru,Санкт-Петербург,Санкт-Петербург,59.938784,30.314997,Прочие,Технический писатель,Технический писатель,<p><strong>Обязанности:</strong></p> <ul> <li>...,полный день,от 1 года,не имеет значения,93862,110507,PERCo,2025-04-28,https://hh.ru/vacancy/118721989
7624,hh.ru,Санкт-Петербург,Санкт-Петербург,59.938784,30.314997,Прочие,Технический писатель,Начинающий специалист по разработке техническо...,<p><strong>Компания ООО &quot;НПП &quot;Адвент...,полный день,без опыта,не имеет значения,50000,50000,Адвент НПП,2025-04-25,https://hh.ru/vacancy/119968837


In [31]:
#Сохраняем модель и RMSE
# joblib.dump(model_ML_salary, "salary_model_cb.pkl")
# joblib.dump(rmse, "rmse_cb.pkl")

model_ML_salary.save_model("model_ML_salary.cbm")

In [ ]:
# model_ML_salary = joblib.load("C:\\Users\\velvi\\Marta\\ITMO\\projects\\graduate work\\salary_model_cb.pkl")
# loaded_rmse = joblib.load("C:\\Users\\velvi\\Marta\\ITMO\\projects\\graduate work\\rmse_cb.pkl")

# rmse = joblib.load("rmse_cb.pkl")